In [2]:
import cv2
from keras.backend import  dropout, relu, softmax
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,Activation,MaxPooling2D,Dense,BatchNormalization,Dropout,Flatten

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.python.keras.engine import input_spec
from tensorflow.python.keras.layers.core import Dropout

In [3]:
from keras.preprocessing.image import img_to_array


In [4]:
img_rows=48
img_cols=48

In [5]:
train=ImageDataGenerator(rescale=1./225).flow_from_directory(
        directory='train',
        batch_size=8,
        target_size=(48,48),
        class_mode="categorical",
        color_mode='grayscale',    
    )

test=ImageDataGenerator(rescale=1./225).flow_from_directory(
        directory='validation',
        batch_size=8,
        target_size=(48,48),
        class_mode="categorical",
        color_mode='grayscale',  
    ) 


Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [6]:
model = Sequential()



model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))



model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))



model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))



model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))



model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))



model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))



model.add(Dense(5,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
                train,
                steps_per_epoch=24282//8,
                epochs=25,
                validation_data=test,
                validation_steps=5937//8)


In [ ]:
model.save_weights('emotion.h5')

In [9]:
model.load_weights('emotion.h5')

In [10]:
import requests
import imutils

In [11]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default (1).xml')
class_labels = ['Angry','Happy','Normal','Sad','Surprise']
cap = cv2.VideoCapture(0)
url="http://192.168.0.100:8080/shot.jpg"
while True:
    #ret, img = cap.read()
     img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    labels = []

    
    faces = face_classifier.detectMultiScale(gray,1.3,3)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)
            preds = model.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position = (50,50)
            cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(225,0,0),2)
        else:
            cv2.putText(img,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow('Emotion Detector',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

